**Pouya Sadeghi**

fall-1401(2022)

# AI course, project #1
Informed and uninformed Search

# 01.Description

## Goals:
**become familiar with**:
- Uninformed search algorithms, such as:
    - BFS
    - IDS
- Informed search algorithms, such as:
    - A*
    - weighted A*
    
and also be able to compare the above items and tell
the advantages and disadvantages of each.

## about world:

in given graph(world), there are 2 types of nodes:
- normal node: regular nodes
- impassible node: they can hold you more than 1 step

also, each node could contain:
- morid(consumer): ordered special meal
- recipe: need them for special meals

## model problem

**State**:

we define each state by
- current location
- collected recipes
- satisfied morids
- how much downtime is left

it contains additional features for computation simplicity.
we might use another feature because of implementation or for predict result faster.
***
**Initial State**:

placed in current node;
rest mush be calculated.
***
**Actions**:

it can move to each of adjacent or stay in current place until it can move.
`(stay and reduce remaining time| move to neighbor)`
***
**Transition Model**:

stay in current place to reduce remaining time or move to one of neighbor.
it can change state via:
1) change current place
2) change collected recipe
3) change satisfied consumer number
4) change remaining time
***
**Goal State**:

satisfying all morids by collecting required recipe and delivering the desired meal to them.
***
**attention**: our cost is the amount of time taken to reach the Goal State.

# 02.Initialization

**Attention**!!

all codes are available in `search` directory.

In [1]:
import timeit

In [2]:
import search
import search.utils as util

## find testcases

In [3]:
TESTS_DIRECTORY = "./testcases"
TEST_FILE_EXTENSION = ".txt"

In [4]:
test_files = util.find_files_with_extension(TESTS_DIRECTORY, TEST_FILE_EXTENSION)

## generate worlds

In [5]:
worlds = [search.initialize_from_file(file) for file in test_files]

In [6]:
for i, w in enumerate(worlds, start=1):
    print(f"world #{i}")
    print(w)
    print("_" * 80)

del i, w

world #1
Graph with 11 nodes and 11 edges
Hard nodes: [9]
Recipes: {7: [4, 10]}
Consumers: [7]
Start node: 0
________________________________________________________________________________
world #2
Graph with 30 nodes and 30 edges
Hard nodes: [15]
Recipes: {27: [2, 10, 12], 28: [4], 6: [23, 8]}
Consumers: [27, 28, 6]
Start node: 27
________________________________________________________________________________
world #3
Graph with 50 nodes and 50 edges
Hard nodes: [11]
Recipes: {24: [46], 15: [37], 18: [47], 8: [30]}
Consumers: [24, 15, 18, 8]
Start node: 39
________________________________________________________________________________
world #4
Graph with 13 nodes and 13 edges
Hard nodes: [1, 0]
Recipes: {9: [4, 9, 2], 3: [2, 10, 1], 10: [0, 11, 2]}
Consumers: [9, 3, 10]
Start node: 12
________________________________________________________________________________
world #5
Graph with 11 nodes and 11 edges
Hard nodes: [9]
Recipes: {7: [4, 10]}
Consumers: [7]
Start node: 0
__________

## utils for test execution time

In [7]:
def calc_exec_time(func: callable, repeat: int = 3) -> callable:
    def measurable(*args, **kwargs):
        total_time = 0
        result = None
        for _ in range(repeat):
            st = timeit.default_timer()
            result = func(*args, **kwargs)
            total_time += timeit.default_timer() - st
        total_time = total_time / repeat
        return total_time, result
    return measurable

In [8]:
def run_search_algorithm(measurable_algorithm: callable, *args, **kwargs):
    exec_time, res = measurable_algorithm(*args, **kwargs)
    print(f"- exec time: {exec_time*1000:.5f} (ms)")
    print(f"- path cost: {res[0].time}")
    print(f"- visited states: {res[1]}")
    print(f"- path: {'->'.join(str(x) for x in res[2])}")
    print()
    return exec_time, res

In [9]:
def run_search_on_worlds(test_worlds, search_algorithm, **alg_args):
    df = util.create_df()
    measurable = calc_exec_time(search_algorithm)
    for n, world in enumerate(test_worlds, start=1):
        print(f"world #{n}")
        exec_time, res = run_search_algorithm(
            measurable,
            **{
                key: ev(world) for key, ev in alg_args.items()
            }
        )
        df = util.add_to_df(df, res[0], res[1], res[2], exec_time)
    return df

# 3.Algorithms

## BFS

- it's an uninformed search algorithm and explore states based on depth(time in this case)
- advantages:
    - complete
    - optimal
    - better exec-time than IDS
- disadvantages:
    - time (space) complexity
- time complexity: $O(b^d)$
- space complexity: $O(b^d)$


    d: depth of optimal solution

    b: maximum branching factor

In [10]:
bfs_df = run_search_on_worlds(
    worlds,
    search.bfs,
    start_state=lambda w: w["start state"],
    goal_test=lambda w: w["goal tester"],
    transition_function=lambda w: w["transition function"]
)
bfs_df

world #1
- exec time: 0.21018 (ms)
- path cost: 8
- visited states: 37
- path: 1->3->4->5->7->10->11->9->8

world #2
- exec time: 866.01242 (ms)
- path cost: 12
- visited states: 56857
- path: 28->23->13->3->11->24->9->22->28->22->5->7->29

world #3
- exec time: 130.23411 (ms)
- path cost: 21
- visited states: 18682
- path: 40->42->38->24->31->45->30->48->41->18->1->19->43->49->47->49->9->34->25->50->12->16

world #4
- exec time: 35.29542 (ms)
- path cost: 13
- visited states: 5772
- path: 13->11->10->3->2->6->12->5->9->4->1->13->11->10

world #5
- exec time: 0.17574 (ms)
- path cost: 8
- visited states: 37
- path: 1->3->4->5->7->10->11->9->8

world #6
- exec time: 0.62336 (ms)
- path cost: 8
- visited states: 132
- path: 9->10->2->4->12->3->7->5->8



,visited_states,result_time,algorithm_time,path
0,37,8,0.000210,"(1, 3, 4, 5, 7, 10, 11, 9, 8)"
1,56857,12,0.866012,"(28, 23, 13, 3, 11, 24, 9, 22, 28, 22, 5, 7, 29)"
2,18682,21,0.130234,"(40, 42, 38, 24, 31, 45, 30, 48, 41, 18, 1, 19..."
3,5772,13,0.035295,"(13, 11, 10, 3, 2, 6, 12, 5, 9, 4, 1, 13, 11, 10)"
4,37,8,0.000176,"(1, 3, 4, 5, 7, 10, 11, 9, 8)"
5,132,8,0.000623,"(9, 10, 2, 4, 12, 3, 7, 5, 8)"


## IDS

- it's an uninformed search algorithm and explore states using DFS approach with limited depth (increasing in each iteration)
- advantages:
    - complete
    - optimal
    - better space usage than DFS
- disadvantages:
    - time complexity
- time complexity: $O(b^d)$
- space complexity: $O(b*d)$


    d: depth of optimal solution

    b: maximum branching factor

**tip**: 
to predict failure, we mapped remained_depth to States

In [11]:
ids_df = run_search_on_worlds(
    worlds,
    search.ids,
    start_state=lambda w: w["start state"],
    goal_test=lambda w: w["goal tester"],
    transition_function=lambda w: w["transition function"]
)
ids_df

world #1
- exec time: 1.34333 (ms)
- path cost: 8
- visited states: 139
- path: 1->3->4->5->7->10->11->9->8

world #2
- exec time: 2341.36240 (ms)
- path cost: 12
- visited states: 124787
- path: 28->19->13->3->11->24->9->23->28->23->5->7->29

world #3
- exec time: 1698.05374 (ms)
- path cost: 21
- visited states: 83383
- path: 40->42->38->24->31->45->30->48->41->18->1->19->43->49->47->49->9->34->25->50->12->16

world #4
- exec time: 119.26418 (ms)
- path cost: 13
- visited states: 15382
- path: 13->11->10->3->2->6->12->5->9->4->1->13->11->10

world #5
- exec time: 1.28633 (ms)
- path cost: 8
- visited states: 139
- path: 1->3->4->5->7->10->11->9->8

world #6
- exec time: 2.53136 (ms)
- path cost: 8
- visited states: 389
- path: 9->10->9->4->12->3->7->5->8



,visited_states,result_time,algorithm_time,path
0,139,8,0.001343,"(1, 3, 4, 5, 7, 10, 11, 9, 8)"
1,124787,12,2.341362,"(28, 19, 13, 3, 11, 24, 9, 23, 28, 23, 5, 7, 29)"
2,83383,21,1.698054,"(40, 42, 38, 24, 31, 45, 30, 48, 41, 18, 1, 19..."
3,15382,13,0.119264,"(13, 11, 10, 3, 2, 6, 12, 5, 9, 4, 1, 13, 11, 10)"
4,139,8,0.001286,"(1, 3, 4, 5, 7, 10, 11, 9, 8)"
5,389,8,0.002531,"(9, 10, 9, 4, 12, 3, 7, 5, 8)"


## A-Star

- it's an informed search algorithm and explore states using dijkstra approach.
this algorithm tries to continue the search in a more targeted way by predicting the amount of cost from the current state to the goal state.

This algorithm uses heuristics to predict the cost of continuing the route.

- features:
    - optimality: depends on heuristic
        - admissible heuristic for tree search
        - consistent heuristic for graph search

    - complete

- time complexity: number of node $f(n) <= C*$
- space complexity: $exponential$


    f(c): cost(n) + heuristic(n)

### heuristic

we defined heuristic-function as minimum time(depth) you have to spend to reach to the goal state from current state.

for this purpose, we calculate each node distance in the graph, assuming cost 1 for each edge, then we calculate minimum path cost.
this wouldn't go higher than real cost; because it is optimal path with minimum cost, discarding impassible nodes(stuck for more tha 1 time unit in a node);

so, it is `admissible`; because
    for every state n, $h(n) <= h*(n)$
and it won't over-estimate the cost to reach the goal.


real cost is grater (when stuck in an impassible node) or equal to the cost implied by heuristic;

so, it's `consistent` and
    $cost(n_1 -> n_2) >= h(n_1) - h(n_2)$

**attention**: $|h(n_1) - h(n_2)| <= 1$ and in each step, heuristic estimation can't change more than 1 unit.

***
There is always a trade-off between accuracy and speed in heuristics.

Here, we have another heuristic (it's also admissible and consistent) which needs less computation than the old one.
this heuristic is based on satisfaction and number of *recipes* collected and *satisfied consumer*.

$h(n) = NumberOfNodeWithConsumerWaitingOrRecipeRemaining$

To try this heuristic, you need to replace `generate_path_base_heuristic` with `generate_satisfaction_base_heuristic` in code-block below.

It is admissible, because our total cost is al least, equal to $NumberOfRecipes + NumberOfConsumers$

Is is consistent, because each state transition cost is $1$ and our heuristic in each transition would $change \in [-1, 0]$,
so due to $|h(n_1) - h(n_2)| <= 1 = cost(n_1 -> n_2)$ it is consistent.

In [12]:
heuristic_generator = search.generate_path_base_heuristic

In [13]:
a_start_df = run_search_on_worlds(
    worlds,
    search.a_star,
    start_state=lambda w: w["start state"],
    goal_test=lambda w: w["goal tester"],
    transition_function=lambda w: w["transition function"],
    heuristic= lambda w: heuristic_generator(w)
)
a_start_df

world #1
- exec time: 0.11644 (ms)
- path cost: 8
- visited states: 10
- path: 1->3->4->5->7->10->11->9->8

world #2
- exec time: 613.26525 (ms)
- path cost: 12
- visited states: 38
- path: 28->22->9->24->11->3->13->23->5->7->29->22->28

world #3
- exec time: 34.50089 (ms)
- path cost: 21
- visited states: 22
- path: 40->42->38->24->31->45->30->48->41->18->1->19->43->49->47->49->9->34->25->50->12->16

world #4
- exec time: 83.07329 (ms)
- path cost: 13
- visited states: 34
- path: 13->11->10->3->2->6->12->5->9->4->1->13->11->10

world #5
- exec time: 0.10081 (ms)
- path cost: 8
- visited states: 10
- path: 1->3->4->5->7->10->11->9->8

world #6
- exec time: 0.47665 (ms)
- path cost: 8
- visited states: 14
- path: 9->10->2->4->12->3->7->5->8



,visited_states,result_time,algorithm_time,path
0,10,8,0.000116,"(1, 3, 4, 5, 7, 10, 11, 9, 8)"
1,38,12,0.613265,"(28, 22, 9, 24, 11, 3, 13, 23, 5, 7, 29, 22, 28)"
2,22,21,0.034501,"(40, 42, 38, 24, 31, 45, 30, 48, 41, 18, 1, 19..."
3,34,13,0.083073,"(13, 11, 10, 3, 2, 6, 12, 5, 9, 4, 1, 13, 11, 10)"
4,10,8,0.000101,"(1, 3, 4, 5, 7, 10, 11, 9, 8)"
5,14,8,0.000477,"(9, 10, 2, 4, 12, 3, 7, 5, 8)"


## Weighted A*

we assign a coefficient to heuristic function (coefficient=1 in ordinary A*)

- advantages:
    - it is faster than ordinary A*

- disadvantages:
    - it may not be optimal

by increasing coefficient, execution time would decrease; instead, the deviation from the optimal result may increase.

In [14]:
def weighted_a_star(worlds, weight: float):
    return run_search_on_worlds(
        worlds,
        search.a_star,
        start_state=lambda w: w["start state"],
        goal_test=lambda w: w["goal tester"],
        transition_function=lambda w: w["transition function"],
        heuristic= lambda w: heuristic_generator(w, weight)
    )

### $\alpha=1.7$

In [15]:
w1_a_star_df = weighted_a_star(worlds, 1.7)
w1_a_star_df

world #1
- exec time: 0.08953 (ms)
- path cost: 8
- visited states: 9
- path: 1->3->4->5->7->10->11->9->8

world #2
- exec time: 342.94964 (ms)
- path cost: 12
- visited states: 13
- path: 28->19->13->3->11->24->9->2->5->7->29->22->28

world #3
- exec time: 35.35981 (ms)
- path cost: 21
- visited states: 22
- path: 40->42->38->24->31->45->30->48->41->18->1->19->43->49->47->49->9->34->25->50->12->16

world #4
- exec time: 62.12963 (ms)
- path cost: 14
- visited states: 21
- path: 13->1->4->9->5->12->6->2->3->10->11->13->1->4

world #5
- exec time: 0.10179 (ms)
- path cost: 8
- visited states: 9
- path: 1->3->4->5->7->10->11->9->8

world #6
- exec time: 0.33596 (ms)
- path cost: 9
- visited states: 10
- path: 9->4->12->3->11->2->10->8->5->7



,visited_states,result_time,algorithm_time,path
0,9,8,0.000090,"(1, 3, 4, 5, 7, 10, 11, 9, 8)"
1,13,12,0.342950,"(28, 19, 13, 3, 11, 24, 9, 2, 5, 7, 29, 22, 28)"
2,22,21,0.035360,"(40, 42, 38, 24, 31, 45, 30, 48, 41, 18, 1, 19..."
3,21,14,0.062130,"(13, 1, 4, 9, 5, 12, 6, 2, 3, 10, 11, 13, 1, 4)"
4,9,8,0.000102,"(1, 3, 4, 5, 7, 10, 11, 9, 8)"
5,10,9,0.000336,"(9, 4, 12, 3, 11, 2, 10, 8, 5, 7)"


### $\alpha=3.2$

In [16]:
w2_a_star_df = weighted_a_star(worlds, 3.2)
w2_a_star_df

world #1
- exec time: 0.11211 (ms)
- path cost: 8
- visited states: 9
- path: 1->3->4->5->7->10->11->9->8

world #2
- exec time: 356.43742 (ms)
- path cost: 12
- visited states: 13
- path: 28->19->13->3->11->24->9->2->5->7->29->22->28

world #3
- exec time: 36.50613 (ms)
- path cost: 21
- visited states: 22
- path: 40->42->38->24->31->45->30->48->41->18->1->19->43->49->47->49->9->34->25->50->12->16

world #4
- exec time: 61.76929 (ms)
- path cost: 15
- visited states: 20
- path: 13->1->4->9->2->3->10->5->12->5->10->11->13->1->4

world #5
- exec time: 0.08761 (ms)
- path cost: 8
- visited states: 9
- path: 1->3->4->5->7->10->11->9->8

world #6
- exec time: 0.34104 (ms)
- path cost: 9
- visited states: 10
- path: 9->4->12->3->11->2->10->8->5->7



,visited_states,result_time,algorithm_time,path
0,9,8,0.000112,"(1, 3, 4, 5, 7, 10, 11, 9, 8)"
1,13,12,0.356437,"(28, 19, 13, 3, 11, 24, 9, 2, 5, 7, 29, 22, 28)"
2,22,21,0.036506,"(40, 42, 38, 24, 31, 45, 30, 48, 41, 18, 1, 19..."
3,20,15,0.061769,"(13, 1, 4, 9, 2, 3, 10, 5, 12, 5, 10, 11, 13, ..."
4,9,8,0.000088,"(1, 3, 4, 5, 7, 10, 11, 9, 8)"
5,10,9,0.000341,"(9, 4, 12, 3, 11, 2, 10, 8, 5, 7)"


# 4.Conclusion

## testcase/input1.txt

|Algorithm|answer(minimum time needed)|explored state|execution time(ms)|
|--|--|--|--|
|BFS|8|37|0.21|
|IDS|8|139|1.33|
|A*|8|10|0.12|
|weighted A* $(\alpha=1.7)$|8|9|0.09|
|weighted A* $(\alpha=3.2)$|8|9|0.11|

## testcase/input2.txt

|Algorithm|answer(minimum time needed)|explored state|execution time(ms)|
|--|--|--|--|
|BFS|12|56857|866.01|
|IDS|12|124787|2341.36|
|A*|12|38|613.26|
|weighted A* $(\alpha=1.7)$|12|13|342.95|
|weighted A* $(\alpha=3.2)$|12|13|356.44|

## testcase/input3.txt

|Algorithm|answer(minimum time needed)|explored state|execution time(ms)|
|--|--|--|--|
|BFS|21|18682|130.23|
|IDS|21|83383|1698.05|
|A*|21|22|34.50|
|weighted A* $(\alpha=1.7)$|21|22|35.36|
|weighted A* $(\alpha=3.2)$|21|22|36.50|

## AutoGeneratedTables

Because of changing time in each run, you could use auto-generated tables

In [17]:
tests_names = (
    name.removeprefix(TESTS_DIRECTORY).
    removesuffix(TEST_FILE_EXTENSION).
    replace("/","").replace("\\","") for name in test_files
)
results = util.convert_to_world_df(
    save_to="result.xlsx",
    names=tests_names,
    BFS=bfs_df,
    IDS=ids_df,
    A_STAR=a_start_df,
    A_START_alpha1=w1_a_star_df,
    A_START_alpha2=w2_a_star_df,
)

In [18]:
util.display_df_as_html(*results)

algorithm name,visited_states,result_time,algorithm_time,path
BFS,37,8,0.00021,"(1, 3, 4, 5, 7, 10, 11, 9, 8)"
IDS,139,8,0.001343,"(1, 3, 4, 5, 7, 10, 11, 9, 8)"
A_STAR,10,8,0.000116,"(1, 3, 4, 5, 7, 10, 11, 9, 8)"
A_START_alpha1,9,8,0.00009,"(1, 3, 4, 5, 7, 10, 11, 9, 8)"
A_START_alpha2,9,8,0.000112,"(1, 3, 4, 5, 7, 10, 11, 9, 8)"


algorithm name,visited_states,result_time,algorithm_time,path
BFS,56857,12,0.866012,"(28, 23, 13, 3, 11, 24, 9, 22, 28, 22, 5, 7, 29)"
IDS,124787,12,2.341362,"(28, 19, 13, 3, 11, 24, 9, 23, 28, 23, 5, 7, 29)"
A_STAR,38,12,0.613265,"(28, 22, 9, 24, 11, 3, 13, 23, 5, 7, 29, 22, 28)"
A_START_alpha1,13,12,0.34295,"(28, 19, 13, 3, 11, 24, 9, 2, 5, 7, 29, 22, 28)"
A_START_alpha2,13,12,0.356437,"(28, 19, 13, 3, 11, 24, 9, 2, 5, 7, 29, 22, 28)"


algorithm name,visited_states,result_time,algorithm_time,path
BFS,18682,21,0.130234,"(40, 42, 38, 24, 31, 45, 30, 48, 41, 18, 1, 19, 43, 49, 47, 49, 9, 34, 25, 50, 12, 16)"
IDS,83383,21,1.698054,"(40, 42, 38, 24, 31, 45, 30, 48, 41, 18, 1, 19, 43, 49, 47, 49, 9, 34, 25, 50, 12, 16)"
A_STAR,22,21,0.034501,"(40, 42, 38, 24, 31, 45, 30, 48, 41, 18, 1, 19, 43, 49, 47, 49, 9, 34, 25, 50, 12, 16)"
A_START_alpha1,22,21,0.03536,"(40, 42, 38, 24, 31, 45, 30, 48, 41, 18, 1, 19, 43, 49, 47, 49, 9, 34, 25, 50, 12, 16)"
A_START_alpha2,22,21,0.036506,"(40, 42, 38, 24, 31, 45, 30, 48, 41, 18, 1, 19, 43, 49, 47, 49, 9, 34, 25, 50, 12, 16)"


algorithm name,visited_states,result_time,algorithm_time,path
BFS,5772,13,0.035295,"(13, 11, 10, 3, 2, 6, 12, 5, 9, 4, 1, 13, 11, 10)"
IDS,15382,13,0.119264,"(13, 11, 10, 3, 2, 6, 12, 5, 9, 4, 1, 13, 11, 10)"
A_STAR,34,13,0.083073,"(13, 11, 10, 3, 2, 6, 12, 5, 9, 4, 1, 13, 11, 10)"
A_START_alpha1,21,14,0.06213,"(13, 1, 4, 9, 5, 12, 6, 2, 3, 10, 11, 13, 1, 4)"
A_START_alpha2,20,15,0.061769,"(13, 1, 4, 9, 2, 3, 10, 5, 12, 5, 10, 11, 13, 1, 4)"


algorithm name,visited_states,result_time,algorithm_time,path
BFS,37,8,0.000176,"(1, 3, 4, 5, 7, 10, 11, 9, 8)"
IDS,139,8,0.001286,"(1, 3, 4, 5, 7, 10, 11, 9, 8)"
A_STAR,10,8,0.000101,"(1, 3, 4, 5, 7, 10, 11, 9, 8)"
A_START_alpha1,9,8,0.000102,"(1, 3, 4, 5, 7, 10, 11, 9, 8)"
A_START_alpha2,9,8,0.000088,"(1, 3, 4, 5, 7, 10, 11, 9, 8)"


algorithm name,visited_states,result_time,algorithm_time,path
BFS,132,8,0.000623,"(9, 10, 2, 4, 12, 3, 7, 5, 8)"
IDS,389,8,0.002531,"(9, 10, 9, 4, 12, 3, 7, 5, 8)"
A_STAR,14,8,0.000477,"(9, 10, 2, 4, 12, 3, 7, 5, 8)"
A_START_alpha1,10,9,0.000336,"(9, 4, 12, 3, 11, 2, 10, 8, 5, 7)"
A_START_alpha2,10,9,0.000341,"(9, 4, 12, 3, 11, 2, 10, 8, 5, 7)"
